## run this cell before starting spark cluster connection (corrects module error)

In [1]:
# spark.jars.packages com.databricks:spark-avro_2.11:4.0.0
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

### select the data

In [4]:
df_lookup = spark.read.format("com.databricks.spark.avro").load("/user/rucio01/dumps/2018-04-30/contents")
df_april = spark.read.json("/user/sprenner/training_official_april.json")

In [5]:
#df_lookup.limit(2).toPandas()

In [6]:
#df_april.limit(2).toPandas()
df_april.createOrReplaceTempView("training")

In [9]:
df_distinct1 = spark.sql("SELECT datasetScope, COUNT(datasetScope)\
                   FROM training\
                   GROUP BY datasetScope")
df_distinct1.toPandas()

datasetScope  count(datasetScope)
0   data18_comm   6                  
1   mc12_8TeV     13                 
2   data15_hi     6                  
3   mc15_13TeV    120783             
4   data16_13TeV  223671             
5   mc15_8TeV     2                  
6   mc16_5TeV     470                
7   mc15_valid    23188              
8   data15_13TeV  39816              
9   mc15_14TeV    30743              
10  mc16_13TeV    235328             
11  mc15_5TeV     24897              
12  data17_5TeV   552                
13  data17_13TeV  62545              
14  data16_cos    2                  
15  data10_hi     1                  
16  data18_13TeV  11440              
17  mc16_5p44TeV  2635               
18  hc_test       34                 
19  data18_cos    1                  
20  mc16_valid    874                
21  data15_5TeV   4                  
22  mc15_pPb8TeV  275

In [10]:
df_distinct2 = spark.sql("SELECT scope, COUNT(scope)\
                   FROM training\
                   GROUP BY scope")
df_distinct2.toPandas()

scope  count(scope)
0   data18_comm   6           
1   mc12_8TeV     13          
2   data15_hi     6           
3   mc15_13TeV    120056      
4   data16_13TeV  223671      
5   mc15_8TeV     2           
6   mc16_5TeV     470         
7   mc15_valid    23188       
8   data15_13TeV  39816       
9   mc15_14TeV    30743       
10  mc16_13TeV    236089      
11  mc15_5TeV     24897       
12  data17_5TeV   552         
13  data17_13TeV  62545       
14  data16_cos    2           
15  data10_hi     1           
16  data18_13TeV  11440       
17  mc16_5p44TeV  2635        
18  data18_cos    1           
19  mc16_valid    874         
20  data15_5TeV   4           
21  mc15_pPb8TeV  275

In [13]:
#df_lookup = df_lookup.where("DID_TYPE = 'D'")

In [19]:
df_april.agg(*(countDistinct(col(c)).alias(c) for c in ['filename'])).toPandas()

filename
0  530100

In [20]:
df_april = df_april.withColumn("join_key_april", concat_ws(':', df_april['scope'], df_april['filename']))
df_lookup = df_lookup.withColumn("join_key_lookup", concat_ws(':', df_lookup["CHILD_SCOPE"], df_lookup["CHILD_NAME"]))

In [21]:
df_april.agg(*(countDistinct(col(c)).alias(c) for c in ['join_key_april'])).toPandas()

join_key_april
0  530100

In [22]:
df_joined = df_april.join(df_lookup, df_april.join_key_april == df_lookup.join_key_lookup, "left_outer")

In [23]:
df_joined.createOrReplaceTempView("joined")

In [24]:
df_joined.agg(*(countDistinct(col(c)).alias(c) for c in ['join_key_april', 'join_key_lookup'])).toPandas()

join_key_april  join_key_lookup
0  530100          519585

In [25]:
df_joined.limit(10).toPandas()

account  accountHash clientState  clientStateHash  \
0  jsabater  1852428352   DONE       -878527175         
1  jsabater  1852428352   DONE       -878527175         
2  jsabater  1852428352   DONE       -878527175         
3  jsabater  1852428352   DONE       -878527175         
4  jsabater  1852428352   DONE       -878527175         
5  jsabater  1852428352   DONE       -878527175         
6  jsabater  1852428352   DONE       -878527175         
7  jsabater  1852428352   DONE       -878527175         
8  jsabater  1852428352   DONE       -878527175         
9  bochen    983621989    Exception   634051254         

                                                                                               dataset  \
0  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768   
1  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768   
2  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768   
3  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7709   
4  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768   
5  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768   
6  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768   
7  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768   
8  mc15_14TeV.407099.PowhegPythia8EvtGen_A14_ttbar_hdamp172p5_MET100.recon.AOD.e4906_s2630_s2206_r7768   
9  mc16_13TeV.361027.Pythia8EvtGen_A14NNPDF23LO_jetjet_JZ7W.merge.AOD.e3668_s3126_r9364_r9315            

   datasetHash datasetScope  datasetScopeHash          dataset_version  \
0 -1777414577   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7768   
1 -1777414577   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7768   
2 -1777414577   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7768   
3  1880182958   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7709   
4 -1777414577   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7768   
5 -1777414577   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7768   
6 -1777414577   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7768   
7 -1777414577   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7768   
8 -1777414577   mc15_14TeV  -2057006107        e4906_s2630_s2206_r7768   
9  801314966    mc16_13TeV  -1447489496        e3668_s3126_r9364_r9315   

   dataset_versionHash datatype  datatypeHash eventType  eventTypeHash  \
0 -1200349912           AOD      1206945951    download  59378080        
1 -1200349912           AOD      1206945951    download  59378080        
2 -1200349912           AOD      1206945951    download  59378080        
3  1352491158           AOD      1206945951    download  59378080        
4 -1200349912           AOD      1206945951    download  59378080        
5 -1200349912           AOD      1206945951    download  59378080        
6 -1200349912           AOD      1206945951    download  59378080        
7 -1200349912           AOD      1206945951    download  59378080        
8 -1200349912           AOD      1206945951    download  59378080        
9 -580251480            AOD      1206945951    download  59378080        

  eventVersion  eventVersionHash                          filename  \
0  1.14.11     -2044360925        AOD.08047211._000539.pool.root.1   
1  1.14.11     -2044360925        AOD.08047211._000563.pool.root.1   
2  1.14.11     -2044360925        AOD.08047211._000889.pool.root.1   
3  1.14.11     -2044360925        AOD.07978428._000337.pool.root.1   
4  1.14.11     -2044360925        AOD.08047211._000076.pool.root.1   
5  1.14.11     -2044360925        AOD.08047211._000317.pool.root.1   
6  1.14.11     -2044360925        AOD.08047211._000629.pool.root.1   
7  1.14.11     -204

In [27]:
df_check = spark.sql("SELECT *\
                   FROM joined\
                   WHERE join_key_lookup == 'None' OR join_key_lookup IS NULL")
df_check.toPandas()

account  accountHash         clientState  clientStateHash  \
0      yliu      1494332205   DONE               -878527175         
1      yliu      1494332205   DONE               -878527175         
2      yliu      1494332205   DONE               -878527175         
3      yliu      1494332205   DONE               -878527175         
4      oabouzei  1586357282   DONE               -878527175         
5      oabouzei  1586357282   DONE               -878527175         
6      oabouzei  1586357282   DONE               -878527175         
7      oabouzei  1586357282   DONE               -878527175         
8      oabouzei  1586357282   DONE               -878527175         
9      oabouzei  1586357282   DONE               -878527175         
10     oabouzei  1586357282   DONE               -878527175         
11     oabouzei  1586357282   DONE               -878527175         
12     oabouzei  1586357282   DONE               -878527175         
13     oabouzei  1586357282   DONE               -878527175         
14     oabouzei  1586357282   DONE               -878527175         
15     oabouzei  1586357282   DONE               -878527175         
16     oabouzei  1586357282   DONE               -878527175         
17     oabouzei  1586357282   DONE               -878527175         
18     oabouzei  1586357282   DONE               -878527175         
19     oabouzei  1586357282   DONE               -878527175         
20     oabouzei  1586357282   DONE               -878527175         
21     oabouzei  1586357282   DONE               -878527175         
22     oabouzei  1586357282   DONE               -878527175         
23     oabouzei  1586357282   DONE               -878527175         
24     oabouzei  1586357282   DONE               -878527175         
25     oabouzei  1586357282   DONE               -878527175         
26     oabouzei  1586357282   DONE               -878527175         
27     oabouzei  1586357282   DONE               -878527175         
28     oabouzei  1586357282   DONE               -878527175         
29     oabouzei  1586357282   DONE               -878527175         
...         ...         ...    ...                      ...         
12179  gtarna    591296153    DONE               -878527175         
12180  pgadow    1992700062   DONE               -878527175         
12181  ggessner  650824671    DONE               -878527175         
12182  pgadow    1992700062   DONE               -878527175         
12183  pgadow    1992700062   DONE               -878527175         
12184  pgadow    1992700062   DONE               -878527175         
12185  ggessner  650824671    DONE               -878527175         
12186  ggessner  650824671    DONE               -878527175         
12187  pgadow    1992700062   DONE               -878527175         
12188  ggessner  650824671    DONE               -878527175         
12189  isiral   -1891614446   DONE               -878527175         
12190  pgadow    1992700062   DONE               -878527175         
12191  isiral   -1891614446   DONE               -878527175         
12192  pgadow    1992700062   DONE               -878527175         
12193  ggessner  650824671    DONE               -878527175         
12194  pgadow    1992700062   DONE               -878527175         
12195  pgadow    1992700062   DONE               -878527175         
12196  pgadow    1992700062   DONE               -878527175         
12197  pgadow    1992700062   DONE               -878527175         
12198  pgadow    1992700062   DONE               -878527175         
12199  pgadow    1992700062   DONE               -878527175         
12200  pgadow    1992700062   DONE               -878527175         
12201  isiral   -1891614446   DONE               -878527175         
12202  pgadow    1992700062   DONE               -878527175         
12203  pgadow    1992700062   DONE               -878527175         
12204  isiral   -1891614446   DONE               -878527175         
12205 